In [8]:
from pathlib import Path
import os

print(os.getcwd())
destination_folder = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation')

os.chdir(destination_folder)

/home/tomas/Projects/histoseg/histoseg/notebooks/Util


## Down-sampling the images


In [25]:
import pyvips

FACTOR = 2

tiff_paths = Path(os.getcwd()).rglob('*.tiff')

for tiff_path in tiff_paths:
    image_name = tiff_path.stem
    new_name = Path(str(image_name) + '_small.tiff')
    new_path = tiff_path.parent / new_name

    print(new_path)
    image = pyvips.Image.tiffload(str(tiff_path), page=FACTOR)
    image.tiffsave(str(new_path), bigtiff=True)


/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-2958-2019-20x-he-4/ns-adenoca-colon-2958-2019-20x-he-4_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-1708-2019-20x-he-2/ns-adenoca-colon-1708-2019-20x-he-2_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-15071-2019-20x-he-10/ns-adenoca-colon-15071-2019-20x-he-10_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-15071-2019-20x-he-6/ns-adenoca-colon-15071-2019-20x-he-6_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-3043-2019-20x-he-a2/ns-adenoca-colon-3043-2019-20x-he-a2_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-15072-2019-20x-he-a3/ns-adenoca-colon-15072-2019-20x-he-a3_small.tiff
/home/tomas/Projects/histoseg/data/Feit_colon-annotation/ns-adenoca-colon-lupos-15071-2019-20x-he-14/ns-adenoca-colon-lupos-15071-2019-20x-he-14_sma

## Extracting image patches

Next, we can extract image patches from these annotations. This is done by function *generate_dataset_from_annotated_slides*.
The basic parameters are path to the data set *data_path_source*, *destination_folder* to save the tiles and *tile_size* (I recommend 256).

If you want to down-sample the image, you can set the *scale* to an integer > 0. Parameter *neighbourhood* is an integer, which specify the amount
of neighborhood patches of size tile_size^2 that will be extracted. The neighbours do not belong to the same class.
Variable *fraction* specifies the fraction of the tiles that will be saved (useful for large data sets).

In [ ]:
from util.data_manipulation_scripts import generate_dataset_from_annotated_slides

tiles_destination = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation_tiles-256')
generate_dataset_from_annotated_slides(dataset_path_source = destination_folder, tiles_destination = tiles_destination,
                                       tile_size = 256, scale = 0, neighborhood = 0, fraction = 1.0)

## Splitting patch data set

The data set of extracted patches can be easily split into training, validation, (and possibly testing) data sets.

In [ ]:
from util.data_manipulation_scripts import split_train_valid

tiles_train = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation_tiles-256-train')
tiles_valid = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation_tiles-256-valid')

# Split size specifies the size of the training data set as a real number between 0 and 1.
split_train_valid(source_dir = tiles_destination, data_train = tiles_train, data_valid = tiles_valid, split_size = 0.8)

## Pre-computing annotation map for segmentation validation data set

For the validation and testing splits, you need to precompute the annotation map. It is recommended to use resolution of 32.

In [ ]:
from util.data_manipulation_scripts import precompute_annotation_map

data_validation = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation_valid/ns-adenoca-colon-15071-2019-20x-he-4')
# It is recommended to process all the images at once
# data_validation = Path('/home/tomas/Projects/histoseg/data/Feit_colon-annotation_valid')

precompute_annotation_map(data_validation, resolution = 32)